In [ ]:
#install packages
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
#load image data
import csv
import time
import requests as req
from PIL import Image
from io import BytesIO
import os
Images=[]
list1=os.listdir("Picasso sample/")
for f1 in list1:
  im=Image.open("Picasso sample/"+f1)
  Images.append(im)

In [ ]:
#import and check pytorch
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

In [ ]:
#Initialize latent space embedding model
import clip
clip.available_models()
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

In [ ]:
#import relevant packages
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
#preprocess images
imgs1=[]
i=0
for im in Images:
  imgs1.append(preprocess(im))
  i=i+1
  if i%10==0:
    print(i)

In [ ]:
#Caculate and save high-dimensional embeddings
image_input = torch.tensor(np.stack(imgs1)).cuda()
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
image_features /= image_features.norm(dim=-1, keepdim=True)
im_features=image_features.cpu().numpy()
np.save("drive/My Drive/clip_embeddings.npy",im_features)

In [ ]:
#Dimension Reduction 
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA
tsne = manifold.TSNE(n_components=2, init='pca', random_state=501, metric='cosine')
pre1_t=tsne.fit_transform(im_features)
x_min, x_max = pre1_t.min(0), pre1_t.max(0)
pre1_norm = (pre1_t - x_min) / (x_max - x_min)

In [ ]:
#Save Dimension Reduction results 
with open('drive/My Drive/human_art/Picasso_sample.csv', 'w', newline='') as file:            
    writer = csv.writer(file)
    writer.writerow(["path",'x','y'])
    for i in range(len(imgs1)):
        writer.writerow(["./static/Picasso sample/"+list1[i],pre1_norm[i][0],pre1_norm[i][1]])